In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Session 2

# Orthonormal Periodic Basis


Let us consider signals sampled on the interval $n \in [0,N-1]$.
The fundamental frequency functions

$$
x_1(n) = A \sin \left( \frac{2\pi  }{N}n \right)  \nonumber
$$
$$
y_1(n) = B \cos \left( \frac{2\pi  }{N}n \right)  \nonumber
$$

are periodic with $x(n+N)=x(n), \ y(n+N)=y(n)$. The 'higher' harmonics are given by

$$
x_k(n) = A \sin \left( \frac{2\pi k }{N}n \right)  \nonumber
$$
$$
y_k(n) = B \cos \left( \frac{2\pi k }{N}n \right)  \nonumber
$$

are periodic with $x(n+\frac{N}{k})=x(n), \ y(n+\frac{N}{k})=y(n)$, i.e. their frequencies are $k$ times higher than the fundamental frequency.

The range of $k$'s is $k=[0,N/2]$ for even $N$ and $k=[0,(N-1)/2]$ for odd. With $k=0$  being the constant function

$$
x_{(N-1)/2}(n) = B \cos \left( \frac{2\pi 0}{N}n \right) = B \nonumber
$$

So in total we have precisely $N$ discrete sampled harmonic functions or vectors if they are visualized as points in $\mathbb{R}^N$. 

$$
(\mathbf{x}_{k})_n = x_{k}(n) \nonumber
$$

In the exercise we will show that harmonics are mutually orthogonal, so when properly normalized and numbered from $0$ to $N-1$

$$
\boldsymbol{u}_{2k} = \mathbf{x}_{k}/\|\mathbf{x}_{k}\| \nonumber
$$

$$
\boldsymbol{u}_{2k+1} = \mathbf{y}_{k}/\|\mathbf{y}_{k}\| \nonumber
$$

we have a complete orthonormal basis set! We can form a \highlight{basis matrix} $\boldsymbol{U}=\left( \boldsymbol{u}_1 \quad \boldsymbol{u}_2 \quad ... \quad \boldsymbol{u}_N \right)$ where $\|\boldsymbol{u}\|=1$.


In [ ]:
## In this snippet you can see how to create the matrix with the basis vector U
## look at the code and try to understand what is going on!

F_s = 121  # sampling frequency
time_length = 1 # duration of the sampling window
N = F_s * time_length # number of samples 

# ensure that we are choosing an odd number of samples
N = int(2*np.floor(N/2)+1)
print(N)

###################
#### BASIS CREATION
###################

indx = np.arange(0,N,1)
U = np.zeros((N,N)) # N x N
U[:,0] = np.ones(N)/np.sqrt(N)  

n_pairs = int((N-1)/2)

for k in range(1,n_pairs + 1):
    # in each column we are going to define cosine or sine function with frequency k
    U[:,2*k] = np.cos(2*(np.pi/N)*(indx.T)*(k)) ## at each step we are filling this is a column
    U[:,2*k] = U[:,2*k]/(np.linalg.norm(U[:,2*k])) # length = sqrt(N / 2)

    U[:,2*k-1] = np.sin(2*(np.pi/N)*(indx.T)*(k))
    U[:,2*k-1] = U[:,2*k-1]/ (np.linalg.norm(U[:,2*k-1]))


### Question 1

Show that the matrix U is orthonormal, i.e. that the vectors are both unit length and they are orthogonal. To do that you should prove that:
1) all vectors of the basis (which in this case are the columns of the matrix U) have length 1;
2) check that two general vectors $U(2k)_j = \sin(2\pi \frac{k}{N}j)$ and $U(2k+1)_j = \cos(2\pi \frac{k'}{N}j)$ are orthogonal. Using the Trigonometric identities, try to show that:
$$ \sum_{j=0}^{N-1}\sin\left(2\pi \frac{k}{N}j\right)\cos\left(2\pi \frac{k'}{N}j\right) = 0 $$
 

### Question 2

In this exercise you should understand what is the effect of a filter to a signal. But how does a filter work? A filter can be seen as a vector of weights. For each sample $x_i$, we consider the previous and the following $n_{\text{neigh}}$ samples, and we compute the sum of the element-wise product between the samples considered and the filter weights. We then substitute $x_i$ with the computed sum. You will understand the meaning of low-pass and high-pass through the examples.

You should create a random signal (look at np.random.randn()) and try to use a low-pass filter and high-pass filter on it. What kind of differences do you notice? 
In addition to this, you can try to use a low-pass and a high-pass filter in sequence. Try to use first the low-pass and then the high-pass and then repeat the experiment with the opposite order. What do you notice?

In [ ]:
########################
####  FILTER CREATION
########################

### you should not modify this function, you can look at that and try to understand how the filter is built.
### printing each step, can help understanding that. 

def create_filter(n_neighbours, low_pass):
    '''
    
    :param n_neighbours: number of sample points consider in both direction (left and right) centered in the 
                         current sample;
    :param low_pass: boolean, if True we create a lowpass filter, otherwise a high-pass filter
    :return: a filter vector
    '''
    my_filter = np.zeros(2*n_neighbours+1)
    my_filter[0:n_neighbours] = np.linspace(0,1,n_neighbours)
    my_filter[(n_neighbours+1):(2*n_neighbours+1)] = np.linspace(1,0,n_neighbours)
    
    if low_pass:
        my_filter[n_neighbours] = my_filter[n_neighbours-1] + my_filter[1]
    else:
        my_filter[n_neighbours]=-2*n_neighbours
    
    my_filter = my_filter/np.linalg.norm(my_filter)
    
    return my_filter



In [ ]:
def compute_filtered_signal(my_filter, signal, n_neigh):
    '''

    :param my_filter: the filter vector (low-pass or high-pass)
    :param signal: a random signal (or N signals)
    :param n_neigh: number of neighbours considered when you built the filter
    :return:
    '''
    ## we recover the number of samples in the signal
    n = signal.shape[0]
    ## we recover the number of signals
    n_signals = signal.shape[1]
    ## we instantiate the signal we want to return
    filtered_signal = np.zeros((n, n_signals))
    for t in range(n):
        # for each sample of the signal
        for tau in range(-n_neigh, n_neigh + 1):
            # we recover all the neighbours we want to consider and we weight it using the filter
            filtered_signal[t,:] += my_filter[tau + n_neigh] * signal[np.remainder(t + tau, n),:]

    return filtered_signal

### You should create the signal (called signal in the code below) and the filter (called filt)
### and you should also define all the parameters needed to being able to sample a signal and create a filter
### Be careful: define a signal as a COLUMN VECTOR. If you are not sure if you did it right, print(your_vector.shape)
##### XXX: Fill me in!





### Question 3

Now that you have understand the effects of applying a filter to a signal, we can go on with the main exercise. If you look at the first code snippet, we have just created our orthonormal basis. Now you should create: M random signals, where you should start with M=1000, where each signal contains N samples, a low pass filter (as you did before), and compute the filtered signals. 

In [ ]:
# Define N, create the (N, M) matrix with all the random signal as column vector. Define a low-pass filter 
# and compute the filtered signals. 

##### XXX: Fill me in!






### Question 4

At this point you should have your (N, M) matrix that contains the filtered signals. Now, as you did last lecture, you are going to project both the original signals and the filtered signals on the periodic function basis and for both of them you are going to compute the variance of each signal in the matrix (this means that you should compute the variance along the columns --> check the parameter **axis** in the function np.var()). 
Remember that the projection can be computed as:
$$ \mathbf{z_{\text{proj}}} = \mathbf{U}^T \mathbf{x}$$

Once you have computed the variances, plot them in the same plot. What do you notice?

-- You can repeat the same exercise considering M=2 instead of M=1000. Discuss the differences in the plots.



In [ ]:
## Take your two matrices with the original signal and the filtered one and project them into the orthonormal basis. 
## Recall that the projection can be done with a matrix multiplication (you can use either np.matmul ior the symbol @).

##### XXX: Fill me in!




### Question 5

In the last part of this exercise session, instead of looking at random signals, we will consider random periodic signals. You should start creating M random periodic functions (we suggest always M=1000) with a certain center frequency and bandwidth, plot only one of these functions and describe it.

In [ ]:
##### XXX: Fill me in!

M= 
f_center = 
bandwidth =

fmin =
fmax = 

# signals creation (you should not modify this function)
# TODO: EXPLAIN WHAT IS GOING ON IN THE NEXT LINE
x = U[:,fmin:fmax] @ np.random.randn(fmax-fmin,M) 

## plot a signal from this matrix
##### XXX: Fill me in!



# now project x on the basis and compute the variance as before. Plot it!
##### XXX: Fill me in!






### Question 6 

Now the signals you should create get more interesting. We define them as a sum of periodic functions centered in two different main centers. In this exercise you should also create the function that generates these periodic signals (Hint: try to look at the function given to you in the previous exercise)

In [ ]:
##### XXX: Fill me in!

M =
f_center1 = 
f_center2 = 
bandwidth =


fmin = 
fmax = 
gmin = 
gmax = 

# Now it is your time to create the signals!
x =

## plot a signal from this matrix
##### XXX: Fill me in!



# now project x on the basis and compute the variance as before. Plot it!
##### XXX: Fill me in!



